In [11]:
import matplotlib.pyplot as plt
import numpy as np

def calculate_sinusoidal_data(params):
    """
    Calculate velocity data for a sinusoidal inflow profile
    
    Parameters:
        params: dict containing velocity profile parameters
    Returns:
        Dictionary with time arrays and velocity data
    """
    # Extract parameters
    u_max = params['u_max']
    dt = params['dt']
    heart_rate = params['heart_rate']  # Heart rate in BPM
    time_range = params['time_range']
    num_points = params['num_points']
    offset = params.get('offset', 1.0)
    amplitude = params.get('amplitude', 1.0)
    
    # Convert heart rate to frequency in Hz
    frequency = heart_rate / 60.0  # BPM to Hz conversion
    
    # Calculate derived values
    omega = 2.0 * np.pi * frequency
    period = 1.0 / frequency
    t = np.linspace(time_range[0], time_range[1], num_points)
    
    # Calculate velocity
    u = u_max * (offset + amplitude * np.sin(omega * t))
    
    return {
        'time': t,
        'velocity': u,
        'period': period,
        'frequency': frequency,
        'heart_rate': heart_rate,
        'timesteps_per_period': period / dt,
        'equation': f"{offset} + {amplitude} * sin(ωt)"
    }

def plot_sinusoidal_profile(data, plot_params=None):
    """
    Create a plot of sinusoidal inflow velocity profile
    
    Parameters:
        data: Dictionary from calculate_sinusoidal_data
        plot_params: Dictionary with plotting parameters
    Returns:
        Matplotlib figure object
    """
    # Default plot parameters
    if plot_params is None:
        plot_params = {}
    
    # Extract plotting parameters
    figsize = plot_params.get('figsize', (12, 6))
    title = plot_params.get('title', 'Sinusoidal Inflow Velocity Profile')
    save_path = plot_params.get('save_path', None)
    dpi = plot_params.get('dpi', 300)
    show_period_lines = plot_params.get('show_period_lines', False)
    show_bpm_annotation = plot_params.get('show_bpm_annotation', True)
    color = plot_params.get('color', 'blue')
    
    # Extract data
    t = data['time']
    u = data['velocity']
    period = data['period']
    heart_rate = data['heart_rate']
    
    # Create plot
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot velocity profile
    ax.plot(t, u, color=color, 
            label=f'u_max * ({data["equation"]})')
    
    # Add period markers if requested
    if show_period_lines and heart_rate <= 600:  # Limit to reasonable heart rates
        for i in range(int(t[-1] / period) + 1):
            if i * period >= t[0] and i * period <= t[-1]:
                ax.axvline(x=i*period, color='gray', linestyle=':', alpha=0.5)
    
    # Labels and formatting
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Velocity (m/s)')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Add heart rate annotation
    if show_bpm_annotation:
        max_y = np.max(u)
        ax.text(0.5 * (t[0] + t[-1]), 0.9 * max_y, 
                f"Heart Rate: {heart_rate} BPM", 
                horizontalalignment='center', 
                bbox=dict(facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    
    # Save if requested
    if save_path:
        plt.savefig(save_path, dpi=dpi)
    
    return fig

# Print simulation statistics
def print_simulation_stats(data):
    """Print statistics about the simulation"""
    print(f"Oscillation period: {data['period']:.4f} seconds")
    print(f"Heart Rate: {data['heart_rate']} BPM")
    print(f"Frequency: {data['frequency']:.4f} Hz")
    print(f"Timesteps per oscillation: {data['timesteps_per_period']:.2f}")

# Example usage
inflow_params = {
    'u_max': 0.04,            # Maximum velocity (m/s)
    'dt': 5e-7,               # Time step (seconds)
    'heart_rate': 60,         # Heart rate in beats per minute (60 BPM = 1 Hz)
    'time_range': (0, 1.0),   # Time range to plot (seconds)
    'num_points': 5000,       # Number of points for plotting
    'offset': 1.0,            # Offset for the sine wave
    'amplitude': 1.0          # Amplitude scaling
}

# Calculate data
inflow_data = calculate_sinusoidal_data(inflow_params)

# Plot configuration
plot_config = {
    'title': 'Sinusoidal Inflow Velocity Profile (Cardiac Simulation)',
    'save_path': 'cardiac_inflow_profile.png',
    'dpi': 300,
    'show_period_lines': True
}

# Generate plot
fig = plot_sinusoidal_profile(inflow_data, plot_config)

# Print statistics
print_simulation_stats(inflow_data)

KeyboardInterrupt: 